In [32]:
from PyPDF2 import PdfReader
import fitz
import pdfplumber
import pandas, numpy
import camelot
import math, numpy

In [11]:
file_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\files"
output_path =  r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\pdf-extractor\output"

In [25]:
# get input file


#get output file

tabular_path = r"\TabularPdf.pdf"
textual_path = r"\TextalPdf.pdf"

In [35]:
import fitz

pdf_path = output_path + tabular_path
output_txt_path = output_path + r"\page_basic.txt"

def analyze_pdf(pdf_path, output_txt_path):
    doc = fitz.open(pdf_path)
    with open(output_txt_path, 'w') as output_file:
        for page_num in range(len(doc)):
            page = doc[page_num]
            output_file.write(f"Page {page_num + 1}:\n")
            output_file.write(f"Width: {page.rect.width}\n")
            output_file.write(f"Height: {page.rect.height}\n")
            font_info = {}
            for text in page.get_text("dict")["blocks"]:
                print(f"\n {text}")
                for line in text.get("lines", []):
                    for span in line.get("spans", []):
                        font = span["font"] #ceil the value
                        size = math.ceil(int(span["size"]))
                        font_info[(font, size)] = font_info.get((font, size), 0) + 1
            for (font, size), count in font_info.items():
                output_file.write(f"Font: {font}, Size: {size}, Count: {count}\n")
            output_file.write("\n")
    doc.close()

analyze_pdf(pdf_path, output_txt_path)



 {'number': 0, 'type': 0, 'bbox': (258.4020080566406, 132.0896759033203, 384.49798583984375, 140.33367919921875), 'lines': [{'spans': [{'size': 6.0, 'flags': 0, 'font': 'Helvetica', 'color': -12220216, 'ascender': 1.0750000476837158, 'descender': -0.29899999499320984, 'text': 'Issuer', 'origin': (258.4020080566406, 138.5396728515625), 'bbox': (258.4020080566406, 132.0896759033203, 274.739990234375, 140.33367919921875)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (258.4020080566406, 132.0896759033203, 274.739990234375, 140.33367919921875)}, {'spans': [{'size': 6.0, 'flags': 0, 'font': 'Helvetica', 'color': -12220216, 'ascender': 1.0750000476837158, 'descender': -0.29899999499320984, 'text': 'Industry', 'origin': (363.156005859375, 138.5396728515625), 'bbox': (363.156005859375, 132.0896759033203, 384.49798583984375, 140.33367919921875)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (363.156005859375, 132.0896759033203, 384.49798583984375, 140.33367919921875)}]}

 {'number': 1, 'type': 0, 'bbox'

In [29]:
required_path = output_path + tabular_path
with pdfplumber.open(required_path) as pdf:
    print(len(pdf.pages))
    page = pdf.pages[7]
    
    # for data in page.extract_words():
    #     print(f"\n {data}")
        
    for data in page.extract_text_lines():
        if data['chars']:
            del data['chars']
            
        print(f"\n{data}")

24

{'text': 'Issuer Industry / Rating % e xo pf oe sq uu ri ety Deri% va tt io ves e Nx% ep to At so u s rn se ee tt o sf', 'x0': 258.4448, 'top': 129.6331, 'x1': 567.9323, 'bottom': 143.23302999999999}

{'text': 'Indian Equity and Equity Related Total 80.98% -47.91% 33.07%', 'x0': 258.4424, 'top': 143.0602, 'x1': 566.9731, 'bottom': 149.5602}

{'text': 'Religare Enterprises Ltd Finance 10.17% 10.17%', 'x0': 258.4424, 'top': 153.47185780300003, 'x1': 567.1953387389999, 'bottom': 158.39705680300005}

{'text': 'Swan Energy Ltd Diversified 3.37% 3.37%', 'x0': 258.4424, 'top': 158.67186780300005, 'x1': 565.7716562149999, 'bottom': 163.5970568030001}

{'text': 'Marathon Nextgen Realty Ltd Realty 2.43% 2.43%', 'x0': 258.4424, 'top': 163.87192780300006, 'x1': 565.577726215, 'bottom': 168.79705680300003}

{'text': 'Wockhardt Ltd Pharmaceuticals & Biotechnology 1.80% 1.80%', 'x0': 258.4424, 'top': 169.071927803, 'x1': 566.4409562149999, 'bottom': 173.99705680300008}

{'text': 'Inox Wind Ltd El